# Simpsons_001

* v001: initial commit

In [20]:
from fastai.imports import *
from os.path import join
from torchtext import vocab, data
from fastai.conv_learner import *

In [36]:
from fastai.nlp import *
from fastai.lm_rnn import *

ModuleNotFoundError: No module named 'spacy'

In [24]:
PATH='./'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

# Note: The student needs to practice her shell skills and prepare her own dataset before proceeding:
# - trn/trn.txt (first 80% of nietzsche.txt)
# - val/val.txt (last 20% of nietzsche.txt)

%ls {PATH}

 all.txt    'Simpsons_get dialogues_v001.ipynb'   txts/
 README.md   Simpsons_v001.ipynb


In [15]:
filename = "all.txt"
dialogues = open(join(PATH,filename),'r', encoding='latin1').read()

In [28]:
trn = dialogues[:int(len(dialogues)*.8)]
val = dialogues[int(len(dialogues)*.8):]
with open(join(TRN, 'trn.txt'), 'w') as f:
    f.write(trn)
with open(join(VAL, 'val.txt'), 'w') as f:
    f.write(val)


In [35]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=8; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

NameError: name 'LanguageModelData' is not defined

In [21]:
class CharSeqStatefulGRU(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.GRU(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))